In [38]:
from maze import Maze
from robot import Robot
from take_n_steps import take_n_steps
from print_maze import print_maze
import numpy as np
import pandas as pd
import sys


# global dictionaries for robot movement and sensing
dir_sensors = {'u': ['l', 'u', 'r'], 'r': ['u', 'r', 'd'],
               'd': ['r', 'd', 'l'], 'l': ['d', 'l', 'u'],
               'up': ['l', 'u', 'r'], 'right': ['u', 'r', 'd'],
               'down': ['r', 'd', 'l'], 'left': ['d', 'l', 'u']}
dir_move = {'u': [0, 1], 'r': [1, 0], 'd': [0, -1], 'l': [-1, 0],
            'up': [0, 1], 'right': [1, 0], 'down': [0, -1], 'left': [-1, 0]}
dir_reverse = {'u': 'd', 'r': 'l', 'd': 'u', 'l': 'r',
               'up': 'd', 'right': 'l', 'down': 'u', 'left': 'r'}

In [50]:
first_run_length = []
second_run_length = []

first1_paths = []
first2_paths = []
second1_paths = []
second2_paths = []

# Create a maze
testmaze = Maze("test_maze_02.txt")

for run in range(10):
    # Intitialize a robot; robot receives info about maze dimensions.
    testrobot = Robot(testmaze.dim, (0,0), 'u')

    # Set the robot in the start position. Note that robot position
    # parameters are independent of the robot itself.
    robot_pos = {'location': [0, 0], 'heading': 'u'}
    
    #find the goal
    while testrobot.move_count < 1000 and not testrobot.found_goal:
        robot_pos = take_n_steps(1, testmaze, testrobot, robot_pos)

        goal_bounds = [testmaze.dim/2 - 1, testmaze.dim/2]
        if robot_pos['location'][0] in goal_bounds and robot_pos['location'][1] in goal_bounds:
            testrobot.found_goal = True
    goal = testrobot.current_location
    first1_paths.append(testrobot.path_taken_to_goal)
    second1_paths.append(a_star((0,0),goal, testrobot.next_locations_table,testmaze.dim))
    
    #find another way to start
    while testrobot.move_count < 2000 and not testrobot.back_to_start:
        robot_pos = take_n_steps(1, testmaze, testrobot, robot_pos)
        if robot_pos['location'][0] == 0 and robot_pos['location'][1] == 0:
            testrobot.back_to_start = True
    first2_paths.append(testrobot.path_taken)
    second2_paths.append(a_star((0,0),goal, testrobot.next_locations_table,testmaze.dim))
        
first1_lengths = [len(x) for x in first1_paths]
first2_lengths = [len(x) for x in first2_paths]
second1_lengths = [len(x) for x in second1_paths]
second2_lengths = [len(x) for x in second2_paths]

scores1 = [first/30 + second for first,second in zip(first1_lengths,second1_lengths)]
scores2 = [first/30 + second for first,second in zip(first2_lengths,second2_lengths)]

results = {"first1":first1_lengths, 
           "second1":second1_lengths,
           "score1":scores1,
           "first2":first2_lengths, 
           "second2":second2_lengths,
           "score2":scores2}

df = pd.DataFrame(results)
print(df[['first1','second1','score1','first2','second2','score2']])

#print(move_counts)
#print(np.mean(move_counts))
#cells = [["   " for x in range(testmaze.dim)] for row in range(testmaze.dim)]     
#cells[testrobot.current_location[0]][testrobot.current_location[1]] = "XXX"
#print_maze(testmaze,cells)

#import get_cells as gc
#cells = gc.get_visited_cells(testrobot,testmaze)
#cells[testrobot.current_location[0]][testrobot.current_location[1]] = "XXX"
#print testrobot.move_count
#print_maze(testmaze,cells)

   first1  second1  score1  first2  second2  score2
0     217       28      35     275       26      35
1     260       28      36     389       24      36
2      46       30      31      90       26      29
3     242       29      37     401       24      37
4      90       30      33     301       26      36
5      67       37      39     118       36      39
6     212       28      35     323       26      36
7     115       28      31     251       28      36
8     150       33      38     241       28      36
9     169       28      33     276       26      35


In [51]:
df.describe()

,first1,first2,score1,score2,second1,second2
count,10.00000,10.000000,10.000000,10.00000,10.000000,10.000000
mean,156.80000,266.500000,34.800000,35.50000,29.900000,27.000000
std,75.51718,100.960113,2.780887,2.54951,2.960856,3.431877
min,46.00000,90.000000,31.000000,29.00000,28.000000,24.000000
25%,96.25000,243.500000,33.000000,35.25000,28.000000,26.000000
50%,159.50000,275.500000,35.000000,36.00000,28.500000,26.000000
75%,215.75000,317.500000,36.750000,36.00000,30.000000,27.500000
max,260.00000,401.000000,39.000000,39.00000,37.000000,36.000000


In [30]:
cells = [["   " for x in range(testmaze.dim)] for row in range(testmaze.dim)]
for x in testrobot.path_taken_to_goal:
    cells[x[0]][x[1]] = " X "
print("steps taken to goal :{}").format(len(testrobot.path_taken_to_goal))
print_maze(testmaze,cells)

steps taken to goal :215
+---+---+---+---+---+---+---+---+---+---+---+---+
| X   X   X   X   X   X   X   X   X   X   X   X |
+---+---    +---+---+---    +---    +---+---    +
| X   X   X | X   X   X   X   X | X   X   X   X |
    +---            +---    +---+---+---    +---+
| X   X | X | X | X   X   X   X | X   X   X   X |
                +---    +---        +---+---+---+
| X | X   X   X | X   X   X   X | X   X   X   X |
    +---+---+---    +---+---+---+---+---        +
| X   X   X | X   X   X   X   X   X   X |   | X |
+---+---            +---+---        +---        +
| X   X   X   X | X |       | X | X   X | X | X |
    +---                                        +
| X   X   X | X   X |     X | X   X | X   X   X |
+---    +---    +---+---    +---        +---    +
|     X   X   X |   | X   X     | X   X | X   X |
    +---+---+---        +---    +---            +
|       |             X   X         | X   X | X |
    +---+---        +---    +---    +---        +
|           |   |        

In [31]:
for x in testrobot.path_taken_to_start:
    cells[x[0]][x[1]] = " O "
print("steps taken to start :{}").format(len(testrobot.path_taken_to_start))
print_maze(testmaze,cells)

steps taken to start :112
+---+---+---+---+---+---+---+---+---+---+---+---+
| O   X   O   X   X   X   X   X   X   X   X   X |
+---+---    +---+---+---    +---    +---+---    +
| O   O   O | X   X   X   X   X | X   X   X   X |
    +---            +---    +---+---+---    +---+
| X   X | X | X | X   X   X   X | X   X   X   X |
                +---    +---        +---+---+---+
| X | X   X   X | X   X   X   X | X   X   X   X |
    +---+---+---    +---+---+---+---+---        +
| O   O   O | X   X   X   X   X   X   X |   | X |
+---+---            +---+---        +---        +
| O   O   O   X | X |     O | X | X   X | X | X |
    +---                                        +
| O   O   O | X   X |     O | X   X | X   X   X |
+---    +---    +---+---    +---        +---    +
| O   O   O   X | O | O   O   O | X   X | X   X |
    +---+---+---        +---    +---            +
| O   O | O   O   O   O   O   O   O | X   X | X |
    +---+---        +---    +---    +---        +
| O   O   O | O | O   O 

In [40]:

def dist_to_finish_estimate(maze_dim):
    heuristic_cost_estimate = np.zeros((maze_dim,maze_dim))
    for x in [i for i in range(-maze_dim/2,maze_dim/2+1) if i != 0]:
        for y in [j for j in range(-maze_dim/2,maze_dim/2+1) if j != 0]:
            if x < 0: 
                dx = maze_dim/2 
            else: 
                dx = maze_dim/2 - 1
            if y < 0:
                dy = maze_dim/2
            else:
                dy = maze_dim/2 - 1
            heuristic_cost_estimate[x+dx][y+dy] = ((abs(x)+abs(y)) - 2)
    return heuristic_cost_estimate

def a_star(start,goal,next_locations, maze_dim):
    closedSet = []
    openSet = [start]
    cameFrom = dict()
    heuristic_cost_estimate = dist_to_finish_estimate(maze_dim)
        
    gScore = {x:maze_dim**2 for x in next_locations.keys()}
    gScore[start] = 0
    fScore = {x:maze_dim**2 for x in next_locations.keys()}
    fScore[start] = heuristic_cost_estimate[start]
    
    def get_min_fscore(locations):
        location_fScore = [fScore[x] for x in locations]
        min_score = min(location_fScore)
        return locations[location_fScore.index(min_score)]
    
    def dist_between(loc1,loc2):
        loc1 = np.array(loc1)
        loc2 = np.array(loc2)
        return sum(abs(loc2-loc1))
    
    while openSet:
        
        current = get_min_fscore(openSet)
        
        if current == goal:
            return reconstruct_path(cameFrom,current)
        
        openSet.remove(current)
        closedSet.append(current)
        
        
        for next_location in next_locations[current]:
            
            if next_location in closedSet:
                continue
            
            if next_location not in openSet:
                openSet.append(next_location)
                    
            tentative_gScore = gScore[current] + 1 #dist_between(current,next_location)
            if tentative_gScore >= gScore[next_location]:
                continue
            
            cameFrom[next_location] = current
            gScore[next_location] = tentative_gScore
            fScore[next_location] = gScore[next_location] + heuristic_cost_estimate[next_location]
        #if current[0] == 4 and current[1] == 4:
         #   location_fScore = [fScore[x] for x in openSet]
          #  print(zip(openSet,location_fScore))
           # print(gScore[current])
    print "Failed"

def reconstruct_path(cameFrom,current):
    total_path = [current]
    while current in cameFrom.keys():
        current = cameFrom[current]
        total_path.append(current)
    return total_path

In [194]:
best_path = a_star(start,goal)
openset = [(0, 1), (0, 2), (0, 4), (0, 7), (0, 8), (0, 9), (1, 5), (1, 4), (1, 3), (6, 3), (4, 5), (4, 6), (4, 7), (4, 3)]
cells = [["   " for x in range(testmaze.dim)] for row in range(testmaze.dim)]
for x in openset:
    cells[x[0]][x[1]] = " 0 "
cells[4][4] = " X "
print_maze(testmaze,cells)

[((0, 1), 10.0), ((0, 8), 10.0), ((0, 9), 11.0), ((1, 4), 10.0), ((6, 3), 10.0), ((4, 3), 11.0), ((4, 5), 9.0), ((4, 6), 9.0), ((4, 7), 10.0)]
7
+---+---+---+---+---+---+---+---+---+---+---+---+
|     0   0       0           0   0   0         |
+---+---    +---+---+---    +---    +---+---    +
|           | 0   0   0         |               |
    +---            +---    +---+---+---    +---+
|       |   |   |               |               |
                +---    +---        +---+---+---+
|   |           |               |               |
    +---+---+---    +---+---+---+---+---        +
|           | 0   X   0   0   0         |   |   |
+---+---            +---+---        +---        +
|               |   |       |   |       |   |   |
    +---                                        +
|           | 0     |       |       |           |
+---    +---    +---+---    +---        +---    +
|               |   |           |       |       |
    +---+---+---        +---    +---            +
|    

In [195]:
import get_cells as gc
cells = gc.get_visited_cells(testrobot,testmaze)
for x in best_path:
    cells[x[0]][x[1]] = " 0 "

print "steps exploring = " + str(testrobot.move_count)
print "best path length = " + str(len(best_path))
print "total score = " + str(testrobot.move_count/30 + len(best_path))
print_maze(testmaze,cells)

steps exploring = 79
best path length = 19
total score = 21
+---+---+---+---+---+---+---+---+---+---+---+---+
| 0           0           0                     |
+---+---    +---+---+---    +---    +---+---    +
|           |     X   X   X     |               |
    +---            +---    +---+---+---    +---+
|       |   |   | X   0   0   X |               |
                +---    +---        +---+---+---+
|   |           | 0   0   X   X |               |
    +---+---+---    +---+---+---+---+---        +
|           | X   X   X   X   X   X     |   |   |
+---+---            +---+---        +---        +
| X   X   X   X | X |       | X | X   X |   |   |
    +---                                        +
| X   X   X | 0   0 |     0 | X   X | X         |
+---    +---    +---+---    +---        +---    +
| 0   X   X   0 |   | 0   0     | X   X |       |
    +---+---+---        +---    +---            +
| 0     |     0       0   X         |       |   |
    +---+---        +---    +---    +---

In [182]:
testrobot = Robot(testmaze.dim, (0,0), 'u')
print(testrobot.U_table)

[[-2.0 -1.8 -1.6 -1.4 -1.2 -1.0 -1.0 -1.2 -1.4 -1.6 -1.8 -2.0]
 [-1.8 -1.6 -1.4 -1.2 -1.0 -0.8 -0.8 -1.0 -1.2 -1.4 -1.6 -1.8]
 [-1.6 -1.4 -1.2 -1.0 -0.8 -0.6 -0.6 -0.8 -1.0 -1.2 -1.4 -1.6]
 [-1.4 -1.2 -1.0 -0.8 -0.6 -0.4 -0.4 -0.6 -0.8 -1.0 -1.2 -1.4]
 [-1.2 -1.0 -0.8 -0.6 -0.4 -0.2 -0.2 -0.4 -0.6 -0.8 -1.0 -1.2]
 [-1.0 -0.8 -0.6 -0.4 -0.2 0.0 0.0 -0.2 -0.4 -0.6 -0.8 -1.0]
 [-1.0 -0.8 -0.6 -0.4 -0.2 0.0 0.0 -0.2 -0.4 -0.6 -0.8 -1.0]
 [-1.2 -1.0 -0.8 -0.6 -0.4 -0.2 -0.2 -0.4 -0.6 -0.8 -1.0 -1.2]
 [-1.4 -1.2 -1.0 -0.8 -0.6 -0.4 -0.4 -0.6 -0.8 -1.0 -1.2 -1.4]
 [-1.6 -1.4 -1.2 -1.0 -0.8 -0.6 -0.6 -0.8 -1.0 -1.2 -1.4 -1.6]
 [-1.8 -1.6 -1.4 -1.2 -1.0 -0.8 -0.8 -1.0 -1.2 -1.4 -1.6 -1.8]
 [-2.0 -1.8 -1.6 -1.4 -1.2 -1.0 -1.0 -1.2 -1.4 -1.6 -1.8 -2.0]]


In [80]:
print(best_path)

[(6, 7), (6, 8), (8, 8), (8, 9), (10, 9), (10, 11), (9, 11), (9, 12), (11, 12), (11, 13), (8, 13), (5, 13), (5, 12), (4, 12), (4, 13), (1, 13), (1, 11), (2, 11), (2, 10), (4, 10), (4, 9), (3, 9), (3, 8), (2, 8), (1, 8), (1, 7), (0, 7), (0, 6), (0, 3), (0, 0)]


In [84]:
next_locations[(1,8)]

[(1, 7), (1, 6), (1, 5), (0, 8), (2, 8)]

In [85]:
next_locations[(2,8)]

[(1, 8), (0, 8), (3, 8)]

In [74]:
print(heuristic_cost_estimate)

[[ 24.  22.  20.  18.  16.  14.  12.  12.  14.  16.  18.  20.  22.  24.]
 [ 22.  20.  18.  16.  14.  12.  10.  10.  12.  14.  16.  18.  20.  22.]
 [ 20.  18.  16.  14.  12.  10.   8.   8.  10.  12.  14.  16.  18.  20.]
 [ 18.  16.  14.  12.  10.   8.   6.   6.   8.  10.  12.  14.  16.  18.]
 [ 16.  14.  12.  10.   8.   6.   4.   4.   6.   8.  10.  12.  14.  16.]
 [ 14.  12.  10.   8.   6.   4.   2.   2.   4.   6.   8.  10.  12.  14.]
 [ 12.  10.   8.   6.   4.   2.   0.   0.   2.   4.   6.   8.  10.  12.]
 [ 12.  10.   8.   6.   4.   2.   0.   0.   2.   4.   6.   8.  10.  12.]
 [ 14.  12.  10.   8.   6.   4.   2.   2.   4.   6.   8.  10.  12.  14.]
 [ 16.  14.  12.  10.   8.   6.   4.   4.   6.   8.  10.  12.  14.  16.]
 [ 18.  16.  14.  12.  10.   8.   6.   6.   8.  10.  12.  14.  16.  18.]
 [ 20.  18.  16.  14.  12.  10.   8.   8.  10.  12.  14.  16.  18.  20.]
 [ 22.  20.  18.  16.  14.  12.  10.  10.  12.  14.  16.  18.  20.  22.]
 [ 24.  22.  20.  18.  16.  14.  12.  12.  14.  16.

In [106]:
def a_star(start,goal):
    closedSet = []
    openSet = [start]
    cameFrom = dict()
    
    gScore = {x:maze_dim**2 for x in visited_locations}
    gScore[start] = 0
    fScore = {x:maze_dim**2 for x in visited_locations}
    fScore[start] = heuristic_cost_estimate[start]
    
    def get_min_fscore(locations):
        location_fScore = [fScore[x] for x in locations]
        min_score = min(location_fScore)
        return locations[location_fScore.index(min_score)]
    
    def dist_between(loc1,loc2):
        loc1 = np.array(loc1)
        loc2 = np.array(loc2)
        return sum(abs(loc2-loc1))
    
    while openSet:
    #for x in range(1):
        current = get_min_fscore(openSet)
        if current == goal:
            return reconstruct_path(cameFrom,current), cameFrom
        openSet.remove(current)
        closedSet.append(current)
        
        for next_location in next_locations[current]:
            #prevent openSet from looping back on itself
            if next_location in closedSet:
                continue
            
            #add next_locations to openSet
            if next_location not in openSet:
                openSet.append(next_location)
            
            tentative_gScore = gScore[current] + dist_between(current,next_location)
            if tentative_gScore >= gScore[next_location]:
                continue
            
            cameFrom[next_location] = current
            gScore[next_location] = tentative_gScore
            fScore[next_location] = gScore[next_location] + heuristic_cost_estimate[next_location]
            
    print "Failed"

def reconstruct_path(cameFrom,current):
    total_path = [current]
    while current in cameFrom.keys():
        current = cameFrom[current]
        total_path.append(current)
    return total_path

In [107]:
best_path,cameFrom = a_star(start,goal)

In [72]:
print(heuristic_cost_estimate)

[[ 24.  22.  20.  18.  16.  14.  12.  12.  14.  16.  18.  20.  22.  24.]
 [ 22.  20.  18.  16.  14.  12.  10.  10.  12.  14.  16.  18.  20.  22.]
 [ 20.  18.  16.  14.  12.  10.   8.   8.  10.  12.  14.  16.  18.  20.]
 [ 18.  16.  14.  12.  10.   8.   6.   6.   8.  10.  12.  14.  16.  18.]
 [ 16.  14.  12.  10.   8.   6.   4.   4.   6.   8.  10.  12.  14.  16.]
 [ 14.  12.  10.   8.   6.   4.   2.   2.   4.   6.   8.  10.  12.  14.]
 [ 12.  10.   8.   6.   4.   2.   0.   0.   2.   4.   6.   8.  10.  12.]
 [ 12.  10.   8.   6.   4.   2.   0.   0.   2.   4.   6.   8.  10.  12.]
 [ 14.  12.  10.   8.   6.   4.   2.   2.   4.   6.   8.  10.  12.  14.]
 [ 16.  14.  12.  10.   8.   6.   4.   4.   6.   8.  10.  12.  14.  16.]
 [ 18.  16.  14.  12.  10.   8.   6.   6.   8.  10.  12.  14.  16.  18.]
 [ 20.  18.  16.  14.  12.  10.   8.   8.  10.  12.  14.  16.  18.  20.]
 [ 22.  20.  18.  16.  14.  12.  10.  10.  12.  14.  16.  18.  20.  22.]
 [ 24.  22.  20.  18.  16.  14.  12.  12.  14.  16.

In [15]:
times_visited = np.zeros((11,11))
times_visited[(0,1)] += 1
print(times_visited)

[[ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]
